In [18]:
import pandas as pd
import sqlite3

import kagglehub

# Download latest version
path = kagglehub.dataset_download("carrie1/ecommerce-data")

print("Path to dataset files:", path)

df = pd.read_csv(path + "/data.csv", encoding='latin-1')

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%m/%d/%Y %H:%M', errors='coerce')

conn = sqlite3.connect("ecommerce.db")
df.to_sql("ecommerce", conn, if_exists="replace", index=False)

df = pd.read_csv("./data.csv")

df.to_sql("countries_references", conn, if_exists="replace", index=False)

query = "PRAGMA table_info(countries_references);"
schema = pd.read_sql(query, conn)

conn.close()

Path to dataset files: /home/aramarchuk/.cache/kagglehub/datasets/carrie1/ecommerce-data/versions/1


Check the structure of the db to ensure that everything is right

In [19]:
conn = sqlite3.connect("ecommerce.db")

query = "PRAGMA table_info(ecommerce);"
schema = pd.read_sql(query, conn)

schema

,cid,name,type,notnull,dflt_value,pk
0,0,InvoiceNo,TEXT,0,None,0
1,1,StockCode,TEXT,0,None,0
2,2,Description,TEXT,0,None,0
3,3,Quantity,INTEGER,0,None,0
4,4,InvoiceDate,TIMESTAMP,0,None,0
5,5,UnitPrice,REAL,0,None,0
6,6,CustomerID,REAL,0,None,0
7,7,Country,TEXT,0,None,0


## Quality Check №1 - Completeness
This check verifies data completeness - it should show us if some essential values are missing. Failure of this check
requires corrective action. Lack of data in columns where values are expected
can lead to incorrect analytics and reporting.

##### Data quality dimension - Completeness

##### SQL logic or condition
`SELECT * FROM ecommerce WHERE customerId is null;`
We can check any important columns in such way, but in this particular table only these fields
contain nulls.

##### Severity level - critical (requires actions)

In [4]:
query = "SELECT * FROM ecommerce WHERE customerId IS NULL;"
schema = pd.read_sql(query, conn)

schema

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536414,22139,NaN,56,12/1/2010 11:52,0.00,None,United Kingdom
1,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,None,United Kingdom
2,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,None,United Kingdom
3,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,None,United Kingdom
4,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,None,United Kingdom
...,...,...,...,...,...,...,...,...
135075,581498,85099B,JUMBO BAG RED RETROSPOT,5,12/9/2011 10:26,4.13,None,United Kingdom
135076,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,12/9/2011 10:26,4.13,None,United Kingdom
135077,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,12/9/2011 10:26,4.96,None,United Kingdom
135078,581498,85174,S/4 CACTI CANDLES,1,12/9/2011 10:26,10.79,None,United Kingdom


In this table we have 135080 rows with unknown CustomerID. I think that should concern us.

## Quality Check №2 - Quantity validation
Typically, quantity is positive, and only in case of cancellation it should be negative.
Cancellation InvoiceNo values are marked with letter 'C' at the beginning.
Any other behavior violates business logic.

##### Data quality dimension - Validity

##### SQL logic or condition
`SELECT * FROM ecommerce WHERE (quantity >= 0 AND InvoiceNo LIKE 'C%') OR (quantity <= 0 AND InvoiceNo NOT LIKE 'C%');`

##### Severity level - critical (requires actions)

In [14]:
query = "SELECT * FROM ecommerce WHERE (quantity >= 0 AND InvoiceNo LIKE 'C%') OR (quantity <= 0 AND InvoiceNo NOT LIKE 'C%');"
schema = pd.read_sql(query, conn)

schema

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536589,21777,NaN,-10,12/1/2010 16:50,0.0,None,United Kingdom
1,536764,84952C,NaN,-38,12/2/2010 14:42,0.0,None,United Kingdom
2,536996,22712,NaN,-20,12/3/2010 15:30,0.0,None,United Kingdom
3,536997,22028,NaN,-20,12/3/2010 15:30,0.0,None,United Kingdom
4,536998,85067,NaN,-6,12/3/2010 15:30,0.0,None,United Kingdom
...,...,...,...,...,...,...,...,...
1331,581210,23395,check,-26,12/7/2011 18:36,0.0,None,United Kingdom
1332,581212,22578,lost,-1050,12/7/2011 18:38,0.0,None,United Kingdom
1333,581213,22576,check,-30,12/7/2011 18:38,0.0,None,United Kingdom
1334,581226,23090,missing,-338,12/8/2011 9:56,0.0,None,United Kingdom


We also have a lot of strange records with negative quantity, but which are not marked as cancellations.
They also don't have description, or have a strange one. Zero UnitPrice, CustomerId.
It really looks like some error in data.

## Quality Check №3 - Uniqueness of InvoiceNo and StockCode
Duplicate data can worsen conclusions, by skewing observations and making them biased.

##### Data quality dimension - Uniqueness

##### SQL logic or condition
```
SELECT * FROM ecommerce
WHERE (InvoiceNo, StockCode) IN (
    SELECT InvoiceNo, StockCode
    FROM ecommerce
    WHERE InvoiceNo IS NOT NULL AND StockCode IS NOT NULL
    GROUP BY InvoiceNo, StockCode
    HAVING COUNT(*) > 1
)
ORDER BY InvoiceNo, StockCode;
```

##### Severity level - warning

In [24]:
# Find all duplicate records
query = """
SELECT * FROM ecommerce
WHERE (InvoiceNo, StockCode) IN (
    SELECT InvoiceNo, StockCode
    FROM ecommerce
    WHERE InvoiceNo IS NOT NULL AND StockCode IS NOT NULL
    GROUP BY InvoiceNo, StockCode
    HAVING COUNT(*) > 1
)
ORDER BY InvoiceNo, StockCode;
"""
duplicates = pd.read_sql(query, conn)

print(f"Found {len(duplicates)} duplicate records")
duplicates.head(20)

Found 20378 duplicate records


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536381,71270,PHOTO CLIP LINE,1,2010-12-01 09:41:00,1.25,15311.0,United Kingdom
1,536381,71270,PHOTO CLIP LINE,3,2010-12-01 09:41:00,1.25,15311.0,United Kingdom
2,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
3,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
4,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
5,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
6,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom
7,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom
8,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom
9,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom


We have some records that completely duplicate InvoiceNo and StockCode, so they probably desscribe
the same transaction. Extra entry should be deleted.

## Quality Check №4 - Timeliness of invoices
Outdated data may reduce analytical relevance and business value, as can future (invalid) dates.
If records fall outside the expected date range, the check can be considered failed.


##### Data quality dimension - Timeliness

##### SQL logic or condition
```
SELECT
    datetime(AVG(julianday(InvoiceDate))) as avg_invoice_date,
    MIN(InvoiceDate) as earliest_date,
    MAX(InvoiceDate) as latest_date,
    COUNT(*) as total_records,
    SUM(CASE WHEN InvoiceDate > '2011-12-10' THEN 1 ELSE 0 END) as records_after_threshold,
    SUM(CASE WHEN InvoiceDate < '2010-12-01' THEN 1 ELSE 0 END) as records_before_threshold
FROM ecommerce
WHERE InvoiceDate IS NOT NULL;
```

##### Severity level - warning

In [23]:
query = """
SELECT
    datetime(AVG(julianday(InvoiceDate))) as avg_invoice_date,
    MIN(InvoiceDate) as earliest_date,
    MAX(InvoiceDate) as latest_date,
    COUNT(*) as total_records,
    SUM(CASE WHEN InvoiceDate > '2011-12-10' THEN 1 ELSE 0 END) as records_after_threshold,
    SUM(CASE WHEN InvoiceDate < '2010-12-01' THEN 1 ELSE 0 END) as records_before_threshold
FROM ecommerce
WHERE InvoiceDate IS NOT NULL;
"""
timeliness_metrics = pd.read_sql(query, conn)
timeliness_metrics

,avg_invoice_date,earliest_date,latest_date,total_records,records_after_threshold,records_before_threshold
0,2011-07-04 13:34:57,2010-12-01 08:26:00,2011-12-09 12:50:00,541909,0,0


## Quality Check №5 - Validity of countries
Countries are represented as strings, so we have to ensure that we can interpret every string, and that we denote
same countries the same way.

##### Data quality dimension - Validity, Consistency
Simple check that country belongs some predefined list of countries is validation check -
"country field is really a country". However, "USA vs United States" would be Consistency check.
Nature of the check is defined by our requirements to the data.

##### SQL logic or condition
```sql
SELECT e.Country, COUNT(*) AS invalid_count
FROM ecommerce e
LEFT JOIN reference_countries r
ON e.Country = r.country_name
WHERE r.country_name IS NULL
GROUP BY e.Country;
```

In [16]:
query = """
    SELECT e.Country, COUNT(*) AS invalid_count
    FROM ecommerce e
    LEFT JOIN countries_references r
    ON e.Country = r.name
    WHERE r.name IS NULL
    GROUP BY e.Country;
"""
timeliness_metrics = pd.read_sql(query, conn)
timeliness_metrics

,Country,invalid_count
0,Channel Islands,758
1,EIRE,8196
2,European Community,61
3,RSA,58
4,USA,291
5,Unspecified,446


USA and RSA we can simply translate to the United States and Republic South Africa (probably).
EIRE is probably Ireland but European Community and Unspecified have to processed separately.

#### Severity level - critical (if we use this data further)